In [31]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
from matplotlib import rcParams,font_manager
import os
from matplotlib.ticker import FormatStrFormatter
import pandas as pd
import pickle
from fourinarowfunctions import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
rcParams['figure.figsize'] = (6,6) #figure size in inches
rcParams['font.family'] = "sans-serif" 
rcParams['font.weight'] = "roman" 
rcParams['font.style'] = "normal" # not 100% sure what this does but it was recommended
rcParams['font.size'] = 32 # not 100% sure what this does but it was recommended
rcParams['pdf.fonttype'] = 42
rcParams['axes.linewidth'] = 2 #thickness of the border
rcParams['xtick.major.width'] = rcParams['axes.linewidth'] # make sure line widths are consistent
rcParams['ytick.major.width'] = rcParams['axes.linewidth']
rcParams['axes.spines.right'] = False #hides right border
rcParams['axes.spines.top'] = False #hides top
rcParams['legend.frameon'] = False #hides box around the legend
rcParams['legend.fontsize'] = 18 #font size in pt
rcParams['axes.labelsize'] = 32 
rcParams['xtick.labelsize'] = 24
rcParams['ytick.labelsize'] = rcParams['xtick.labelsize']
rcParams['lines.linewidth'] = 3
rcParams['xtick.major.size'] = 5
rcParams['lines.markersize'] = 16
rcParams['ytick.major.size'] = rcParams['xtick.major.size'] 

In [15]:
direc = 'C:/Users/svo/Google Drive/Bas Games/Analysis/'
fig_direc = 'C:/Users/svo/Google Drive/Bas Games/Figures/'
peak_direc = 'C:/Users/svo/Documents/FourinarowData/Data/peak/'

In [16]:
def plot_with_errorbars(fig,ax,x,A,colors=None):
    m = np.nanmean(A,axis=0)
    s = np.nanstd(A-m[None,:],axis=0)/np.sqrt(A.shape[0])
    plt.errorbar(x,m,yerr = s,color='black')
    if not colors is None:
        ax.scatter(x,m,color=colors)
    else:
        ax.plot(x,m,'.')

In [17]:
#time_colors = ['#9B1A30','#F85D5F','#FFAC9A']
#learn_colors = ['#4B8F7C', '#3C7864', '#2E614D','#1F4935','#10321D']
#peak_colors = ['#00B4D8','#0096C7','#0077B6','#023E8A','#03045E']

time_colors = ['#9F4AB0','#6E207D','#430F4D']
learn_colors = ['#7DE3AC', '#0EE676', '#129951','#17452D','#20382B']
peak_colors = ['#D7E8FA','#A3CDF7','#3C92E8','#0E5296','#0F3153']

In [18]:
import pandas as pd
boards_hvh = pd.read_csv(direc+'Opt/opt_hvh.txt',delim_whitespace=True,header=None)[[0,1]].values

In [19]:
def load_params_file(filename):
    if os.path.exists(filename):
        return np.loadtxt(filename,delimiter=',')
    else:
        return np.full(shape=[10,],fill_value=np.nan)

def get_peak_user_id(i):
    df = pd.read_csv(peak_direc + 'splits/' + str(i+1) + '/data.csv',delim_whitespace=True,header=None)
    names = pd.unique(df[6])
    assert len(names)==1
    name = names[0]
    return name[:10] + '_block' + name[10:]
    #selected_peak_users = np.unique([player[:player.find('_')] for player in rating_dict.keys() if '_block' in player])

def get_peak_rt(i):
    df = pd.read_csv(peak_direc + 'splits/' + str(i+1) + '/data.csv',delim_whitespace=True,header=None)
    return df[df[4]<10][4].mean()
    #selected_peak_users = np.unique([player[:player.find('_')] for player in rating_dict.keys() if '_block' in player])    

In [20]:
peak_rts = [get_peak_rt(i) for i in range(5000)]
peak_user_ids = [get_peak_user_id(i) for i in range(5000)]

In [21]:
import json
with open(direc + 'Ratings/ratings_all_expts_peak.txt') as f:
    rating_dict = json.load(f)

ratings_learn = np.array([[rating_dict['subj' + str(j).zfill(2) + 'sess' + str(i).zfill(2)] for i in range(5)] for j in range(30)])
ratings_time = np.array([[rating_dict['subj' + str(j).zfill(2) + 'cond' + str(i).zfill(2)] for i in range(3)] for j in range(30)])

In [22]:
#np.savetxt(peak_direc + 'params_peak_final.txt',np.vstack([expand_params(np.loadtxt(peak_direc + 'splits/' + str(i+1) + '/params' + str(j) + '.csv',delimiter=',')) 
#                                              for i in range(5000) for j in range(1,6)]),fmt='%.5f')

In [23]:
ratings_peak = np.reshape(np.array([rating_dict[u] for u in peak_user_ids]),[-1,5])

In [24]:
params_peak = np.array([[expand_params(load_params_file(peak_direc + 'splits/' + str(i+1) + '/params' + str(j+1) + '.csv')) 
                                       for j in range(5)]
                        for i in range(0,5000)])
params_peak = np.reshape(params_peak,[-1,5,5,58]) 
feature_drop_rate_peak = params_peak[:,:,:,-1]

In [25]:
params_learn = np.reshape(np.loadtxt(direc + 'Params/params_learn_final.txt'),[30,5,5,58])
params_time = np.reshape(np.loadtxt(direc + 'Params/params_tai_final.txt'),[30,3,5,58])
feature_drop_rate_learn = params_learn[:,:,:,-1]
feature_drop_rate_time = params_time[:,:,:,-1]


In [26]:

ili_direc = 'C:/Users/svo/Documents/FourinarowData/Data/ili/'
df = pd.read_csv(ili_direc + 'paramsMatrix.csv',header=None)
params_ili_final = np.array([expand_params(r) for r in df.values[:,2:]])
#np.savetxt(direc + 'params_ili_final.txt',params_ili_final)
depth_ili = [np.mean(np.loadtxt(ili_direc + 'depth/depth_ili_final_'+str(i//5)+'_' + str(i%5+1) + '.txt'))
                     for i in range(240)]
heuristic_quality_ili = np.apply_along_axis(get_heuristic_quality,1,params_ili_final)

df['planning depth'] = depth_ili
df['feature drop rate'] = params_ili_final[:,-1]
df['heuristic quality'] = heuristic_quality_ili.flatten()
df.to_csv(ili_direc + 'params_with_metrics.csv')

In [27]:
filename = direc + 'Learning/depth_learn.npy'
if os.path.exists(filename):
    depth_learn = np.load(filename,allow_pickle=True)
else: 
    depth_learn = np.reshape([[np.mean(np.loadtxt(direc + 'Learning/splits/'+str(i)+'/pv_depth_' + str(j) + '.txt'))
                           for j in range(1,6)] for i in range(0,150)],[30,5,5])
    np.save(filename,depth_learn,allow_pickle=True)

In [28]:
filename = direc + 'Timed AI/depth_time.npy'
if os.path.exists(filename):
    depth_time = np.load(filename,allow_pickle=True)
else: 
    depth_time = np.reshape([[np.mean(np.loadtxt(direc + 'Timed AI/splits/'+str(i)+'/pv_depth_' + str(j) + '.txt'))
                           for j in range(1,6)] for i in range(0,90)],[30,3,5])
    np.save(filename,depth_time,allow_pickle=True)

In [29]:
filename = direc + 'Peak/depth_peak.npy'
if os.path.exists(filename):
    depth_peak = np.load(filename,allow_pickle=True)
else: 
    depth_peak = np.reshape([[np.mean(np.loadtxt(peak_direc + 'depth/depth_peak_final_'+str(i)+'_' + str(j) + '.txt'))
                       for j in range(1,6)] for i in range(5000)],[-1,5,5])    
    np.save(filename,depth_peak,allow_pickle=True)

In [30]:
heuristic_quality_learn = np.apply_along_axis(get_heuristic_quality,arr=params_learn,axis=3)
heuristic_quality_time = np.apply_along_axis(get_heuristic_quality,arr=params_time,axis=3)
heuristic_quality_peak = np.apply_along_axis(get_heuristic_quality,arr=params_peak,axis=3)
#heuristic_quality_time = get_heuristic_quality(params_time)
#heuristic_quality_peak = np.vstack([get_heuristic_quality(params_peak[100*i:100*(i+1)]) for i in range(10)])

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,range(1,6),ratings_learn,colors=learn_colors)
ax.set_xlabel('Session')
ax.set_ylabel('Elo rating')
ax.set_xticks(range(1,6))
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_3a.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,range(1,6),np.mean(depth_learn,axis=2),colors=learn_colors)
ax.set_xlabel('Session')
ax.set_ylabel('Planning depth')
ax.set_xticks(range(1,6))
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_3b.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,range(1,6),np.mean(feature_drop_rate_learn,axis=2),colors=learn_colors)
ax.set_xlabel('Session')
ax.set_ylabel('Feature drop rate')
ax.set_xticks(range(1,6))
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_3c.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,range(1,6),np.mean(heuristic_quality_learn,axis=2),colors=learn_colors)
ax.set_xlabel('Session')
ax.set_ylabel('Heuristic quality')
ax.set_xticks(range(1,6))
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_3d.pdf')
plt.show()

In [ ]:
T = np.loadtxt(direc + 'Learning/time_learn.txt')
thinking_time = [np.mean(T[T[:,0]==i,1])/1000 for i in range(150)]

fig,ax=plt.subplots(figsize=(12,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,range(1,6),np.reshape(thinking_time,[30,5]),colors=learn_colors)
ax.set_xlabel('Session')
ax.set_ylabel('Response time(s)')
ax.set_xticks(range(1,6))
ax.set_yticks(np.arange(4,6.5,0.5))
fig.subplots_adjust(bottom=0.15,right=0.98,left=0.12,top=0.95)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'supp_figure_16.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(12,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,np.arange(20,120,20),np.reshape(peak_rts,[1000,5]),colors=peak_colors)
ax.set_xlabel('Games played')
ax.set_xticks(np.arange(20,120,20))
ax.set_ylabel('Response time(s)')
ax.set_yticks(np.arange(2.6,3.4,0.2))
fig.subplots_adjust(bottom=0.15,right=0.98,left=0.12,top=0.95)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'supp_figure_17.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,[5,10,20],ratings_time,colors=time_colors)
ax.set_xlabel('Time limit (s)')
ax.set_ylabel('Elo rating')
ax.set_xticks([5,10,20])
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_3e.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,[5,10,20],np.mean(depth_time,axis=2),colors=time_colors)
ax.set_xlabel('Time limit (s)')
ax.set_ylabel('Planning depth')
ax.set_xticks([5,10,20])
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_3f.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,[5,10,20],np.mean(feature_drop_rate_time,axis=2),colors=time_colors)
ax.set_xlabel('Time limit (s)')
ax.set_ylabel('Feature drop rate')
ax.set_xticks([5,10,20])
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_3g.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,[5,10,20],np.mean(heuristic_quality_time,axis=2),colors=time_colors)
ax.set_xlabel('Time limit (s)')
ax.set_ylabel('Heuristic quality')
ax.set_xticks([5,10,20])
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_3h.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,np.arange(20,120,20),ratings_peak,colors=peak_colors)
ax.set_xlabel('Games played')
ax.set_ylabel('Elo rating')
ax.set_xticks(np.arange(20,120,20))
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_4a.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,np.arange(20,120,20),np.mean(depth_peak,axis=2),colors=peak_colors)
ax.set_xlabel('Games played')
ax.set_ylabel('Planning depth')
ax.set_xticks(np.arange(20,120,20))
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_4b.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,np.arange(20,120,20),np.mean(feature_drop_rate_peak,axis=2),colors=peak_colors)
ax.set_xlabel('Games played')
ax.set_ylabel('Feature drop rate')
ax.set_xticks(np.arange(20,120,20))
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_4c.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,np.arange(20,120,20),np.mean(heuristic_quality_peak,axis=2),colors=peak_colors)
ax.set_xlabel('Games played')
ax.set_ylabel('Heuristic quality')
ax.set_xticks(np.arange(20,120,20))
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'figure_4d.pdf')
plt.show()

In [ ]:
sts.ttest_ind(np.mean(heuristic_quality_learn,axis=2)[:,0],np.mean(heuristic_quality_peak,axis=2)[:,0],equal_var = False)

In [ ]:
np.mean(np.mean(heuristic_quality_learn,axis=2)[:,0]),np.std(np.mean(heuristic_quality_learn,axis=2)[:,0])/np.sqrt(30)

In [ ]:
np.mean(np.mean(heuristic_quality_peak,axis=2)[:,0]),np.std(np.mean(heuristic_quality_peak,axis=2)[:,0])/np.sqrt(1000)

In [ ]:
print('depth')
for x,y in zip([range(5),[5,10,20],range(5)],[depth_learn,depth_time,depth_peak]):
    print(sts.pearsonr(np.tile(x,[y.shape[0],1]).flatten(),np.mean(y,axis=2).flatten()))
    print(sts.spearmanr(np.tile(x,[y.shape[0],1]).flatten(),np.mean(y,axis=2).flatten()))

print('feature drop rate')
for x,y in zip([range(5),[5,10,20],range(5)],[feature_drop_rate_learn,feature_drop_rate_time,feature_drop_rate_peak]):
    print(sts.pearsonr(np.tile(x,[y.shape[0],1]).flatten(),(np.mean(y,axis=2)-np.mean(y,axis=(1,2))[:,None]).flatten()))
    print(sts.spearmanr(np.tile(x,[y.shape[0],1]).flatten(),np.mean(y,axis=2).flatten()))

print('heuristic quality')
for x,y in zip([range(5),[5,10,20],range(5)],[heuristic_quality_learn,heuristic_quality_time,heuristic_quality_peak]):
    print(sts.pearsonr(np.tile(x,[y.shape[0],1]).flatten(),np.mean(y,axis=2).flatten()))
    print(sts.spearmanr(np.tile(x,[y.shape[0],1]).flatten(),np.mean(y,axis=2).flatten()))


In [ ]:
#plt.plot(heuristic_values[0,0,0,player_color==0],optimal_board_values[player_color==0],'.',alpha=0.5)
#plt.plot(heuristic_values[0,0,0,player_color==1],optimal_board_values[player_color==1],'.',alpha=0.5)
#plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(8,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

for i,c in enumerate(learn_colors):
    ax.plot(ratings_learn[:,i],np.mean(depth_learn,axis=2)[:,i],'.',color=c)
for i,c in enumerate(time_colors):
    ax.plot(ratings_time[:,i],np.mean(depth_time,axis=2)[:,i],'.',color=c)

ax.set_xlabel('Elo rating')
ax.set_ylabel('Planning depth')
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.2,top=0.95)
fig.savefig(fig_direc + 'supp_figure_14A.pdf')
plt.show()
sts.pearsonr(np.hstack([ratings_learn,ratings_time]).flatten(),
             np.hstack([np.mean(depth_learn,axis=2),np.mean(depth_time,axis=2)]).flatten())

In [ ]:
fig,ax=plt.subplots(figsize=(8,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

for i,c in enumerate(learn_colors):
    ax.plot(ratings_learn[:,i].flatten(),np.mean(feature_drop_rate_learn,axis=2)[:,i],'.',color=c,alpha=1)
for i,c in enumerate(time_colors):
    ax.plot(ratings_time[:,i].flatten(),np.mean(feature_drop_rate_time,axis=2)[:,i],'.',color=c,alpha=1)

ax.set_xlabel('Elo rating')
ax.set_ylabel('Feature drop rate')
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.2,top=0.95)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'supp_figure_14B.pdf')
plt.show()
sts.pearsonr(np.hstack([ratings_learn,ratings_time]).flatten(),
             np.hstack([np.mean(feature_drop_rate_learn,axis=2),np.mean(feature_drop_rate_time,axis=2)]).flatten())

In [ ]:
fig,ax=plt.subplots(figsize=(8,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

for i,c in enumerate(learn_colors):
    ax.plot(ratings_learn[:,i].flatten(),np.mean(heuristic_quality_learn,axis=2)[:,i],'.',color=c)
for i,c in enumerate(time_colors):
    ax.plot(ratings_time[:,i].flatten(),np.mean(heuristic_quality_time,axis=2)[:,i],'.',color=c)

ax.set_xlabel('Elo rating')
ax.set_ylabel('Heuristic quality')
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.2,top=0.95)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'supp_figure_14C.pdf')
plt.show()
sts.pearsonr(np.hstack([ratings_learn,ratings_time]).flatten(),
             np.hstack([np.mean(heuristic_quality_learn,axis=2),np.mean(heuristic_quality_time,axis=2)]).flatten())

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

for i,c in enumerate(peak_colors):
    ax.plot(ratings_peak[:,i].flatten(),np.mean(depth_peak,axis=2)[:,i],'.',color=c,alpha=0.15)

ax.set_xlabel('Elo rating')
ax.set_ylabel('Planning depth')
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'supp_figure_1b.pdf')
plt.show()
sts.pearsonr(ratings_peak.flatten(),np.mean(depth_peak,axis=2).flatten())

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

for i,c in enumerate(peak_colors):
    ax.plot(np.mean(heuristic_quality_peak,axis=2)[:,i],np.mean(feature_drop_rate_peak,axis=2)[:,i],'.',color=c,alpha=0.15)

ax.set_xlabel('Elo rating')
ax.set_ylabel('Feature drop rate')
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'supp_figure_1b.pdf')
plt.show()
sts.pearsonr(ratings_peak.flatten(),np.mean(feature_drop_rate_peak,axis=2).flatten())

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

for i,c in enumerate(peak_colors):
    ax.plot(ratings_peak[:,i].flatten(),np.mean(heuristic_quality_peak,axis=2)[:,i],'.',color=c,alpha=0.15)

ax.set_xlabel('Elo rating')
ax.set_ylabel('Heuristic quality')
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'supp_figure_1c.pdf')
plt.show()
sts.pearsonr(ratings_peak.flatten(),np.mean(heuristic_quality_peak,axis=2).flatten())

In [ ]:
d1 = np.mean(np.reshape([[np.mean(np.loadtxt(direc + 'Planning depth/depth/depth_hvh_final_' + str(i) + '_' + str(j) + '.txt'))
                           for j in range(1,6)] for i in range(0,40)],[40,5]),axis=1)
d2 = np.mean(np.reshape([[np.mean(np.loadtxt(direc + 'Planning depth/depth/depth_hvh_final2_' + str(i) + '_' + str(j) + '.txt'))
                           for j in range(1,6)] for i in range(0,40)],[40,5]),axis=1)

In [ ]:
f1 = np.mean(np.reshape(np.loadtxt(direc + 'Params/params_hvh_final.txt')[:,-1],[40,5]),axis=1)
f2 = np.mean(np.reshape(np.loadtxt(direc + 'Params/params_hvh_final2.txt')[:,-1],[40,5]),axis=1)

In [ ]:
q1 = np.mean(get_heuristic_quality(np.reshape(np.loadtxt(direc + 'Params/params_hvh_final.txt'),[40,1,5,58])),axis=(1,2))
q2 = np.mean(get_heuristic_quality(np.reshape(np.loadtxt(direc + 'Params/params_hvh_final2.txt'),[40,1,5,58])),axis=(1,2))

In [ ]:
sts.pearsonr(d1,d2),sts.pearsonr(f1,f2),sts.pearsonr(q1,q2)

In [ ]:
def plot_param_bars(rho,sem,names,xlabel,c,filename):
    fig,ax = plt.subplots(figsize=(8,6))
    ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))
    ax.set_xlim([0,1])
    ax.set_xlabel(xlabel,usetex=False)
    ax.barh(range(len(rho),0,-1), rho, xerr=sem, align='center',
        color=c, ecolor='black')
    plt.xticks(np.arange(0,1.2,0.5))
    plt.yticks(range(len(rho),0,-1),names, fontsize =18)
    fig.subplots_adjust(bottom=0.2,right=0.95,left=0.3)
    fig.savefig(fig_direc + filename)
    plt.show()

In [ ]:
plot_param_bars(rho = [sts.pearsonr(d1,d2)[0],sts.pearsonr(f1,f2)[0],sts.pearsonr(q1,q2)[0]],sem=[0,0,0],
                names = ['Planning depth','Feature drop rate','Heuristic quality'],
                xlabel = 'Correlation: repeated fits          ',
                c='red',filename='supp_figure_5A.pdf')

In [ ]:
def plot_correlation_across_conditions(stats,names,xlabel,color,fig_name):
    rho = [np.mean(np.corrcoef(np.mean(x,axis=2).T)[np.triu_indices(x.shape[1],1)]) for x in stats]
    sem = [np.std(np.corrcoef(np.mean(x,axis=2).T)[np.triu_indices(x.shape[1],1)])/np.sqrt(10) for x in stats]
    plot_param_bars(rho,sem,names,xlabel,color,fig_name)
    print(rho,sem,[sts.ttest_1samp(np.corrcoef(np.mean(x,axis=2).T)[np.triu_indices(x.shape[1],1)],popmean=0) for x in stats])

plot_correlation_across_conditions([depth_learn,feature_drop_rate_learn,heuristic_quality_learn],
                               ['Planning depth','Feature drop rate','Heuristic quality'],
                               'Correlation: learning       ',learn_colors[2],'supp_figure_5B.pdf')

In [ ]:
#for name,x in zip(['depth','feature drop rate','heuristic quality'],
#                  [depth_time,feature_drop_rate_time,heuristic_quality_time]):
#    print(name)
#    print('correlation across groups')
#    print(np.corrcoef(np.reshape(x,[90,5]).T))
#    print('correlation across sessions')
#    plt.imshow(np.corrcoef(np.mean(x,axis=2).T),vmin=-1,vmax=1,cmap='bwr')
#    plt.show()
#    print(np.corrcoef(np.mean(x,axis=2).T))

In [ ]:
plt.imshow(np.corrcoef(ratings_peak.T),vmin=-1,vmax=1,cmap='bwr')
plt.show()

for name,x in zip(['depth','feature drop rate','heuristic quality'],
                  [depth_peak,feature_drop_rate_peak,heuristic_quality_peak]):
    print(name)
    print('correlation across groups')
    print(np.corrcoef(np.reshape(x,[-1,5]).T))
    print('correlation across games played')
    plt.imshow(np.corrcoef(np.mean(x,axis=2).T),vmin=-1,vmax=1,cmap='bwr')
    plt.show()

In [ ]:
import pandas as pd
import statsmodels.formula.api as sm

def get_slopes(elo,tree,feat,qual):
    p = sm.ols(formula="y ~ tree + feat + qual",
               data=pd.DataFrame({'y' : elo,'tree': tree,'feat' : feat,'qual' : qual})
              ).fit().params
    return [p[x] for x in ['tree','feat','qual','Intercept']]

In [ ]:
slopes = [get_slopes(ratings_learn[:,i],np.mean(depth_learn,axis=2)[:,i],
                     np.mean(feature_drop_rate_learn,axis=2)[:,i],np.mean(heuristic_quality_learn,axis=2)[:,i])
                 for i in range(5)]
ratings_learn_pred = np.mean(np.sum([s*A for s,A in zip(np.mean(slopes,axis=0),
                                                [depth_learn,feature_drop_rate_learn,heuristic_quality_learn])],axis=0),axis=2)
ratings_learn_pred += np.mean(slopes,axis=0)[-1]

In [ ]:
list(zip(np.mean(slopes,axis=0),np.std(slopes,axis=0)/np.sqrt(5)))

In [ ]:
np.array(slopes)

In [ ]:
print(np.mean(ratings_learn[:,-1]-ratings_learn[:,0]),np.std(ratings_learn[:,-1]-ratings_learn[:,0])/np.sqrt(30))

for s,x in zip(np.mean(slopes,axis=0),[depth_learn,feature_drop_rate_learn,heuristic_quality_learn]):
    print(s,s*np.mean(x[:,-1]-x[:,0]),np.abs(s)*np.std(np.mean(x[:,-1,:]-x[:,0,:],axis=1))/np.sqrt(x.shape[0]))

In [ ]:
slopes = [get_slopes(ratings_peak[:,i],np.mean(depth_peak,axis=2)[:,i],
                     np.mean(feature_drop_rate_peak,axis=2)[:,i],np.mean(heuristic_quality_peak,axis=2)[:,i])
                 for i in range(5)]
ratings_peak_pred = np.mean(np.sum([s*A for s,A in zip(np.mean(slopes,axis=0),
                                                [depth_peak,feature_drop_rate_peak,heuristic_quality_peak])],axis=0),axis=2)
ratings_peak_pred += np.mean(slopes,axis=0)[-1]

In [ ]:
print(np.mean(ratings_peak[:,-1]-ratings_peak[:,0]),np.std(ratings_peak[:,-1]-ratings_peak[:,0])/np.sqrt(1000))

for s,x in zip(np.mean(slopes,axis=0),[depth_peak,feature_drop_rate_peak,heuristic_quality_peak]):
    print(s,s*np.mean(x[:,-1]-x[:,0]),np.abs(s)*np.std(np.mean(x[:,-1,:]-x[:,0,:],axis=1))/np.sqrt(x.shape[0]))

In [ ]:
fig,ax=plt.subplots(figsize=(8,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

plot_with_errorbars(fig,ax,range(1,6),ratings_learn_pred,colors=learn_colors)
#plot_with_errorbars(fig,ax,range(1,6),ratings_learn,colors=learn_colors)
ax.set_xlabel('Session')
ax.set_ylabel('Elo rating')
ax.set_xticks(range(1,6))
fig.subplots_adjust(bottom=0.15,right=0.95,left=0.2,top=0.95)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
fig.savefig(fig_direc + 'supp_figure_14B.pdf')
plt.show()

In [ ]:
1-np.sum((ratings_learn_pred-ratings_learn)**2)/np.sum((ratings_learn-np.mean(ratings_learn))**2)

In [ ]:
sts.pearsonr(ratings_learn_pred.flatten(),ratings_learn.flatten())

In [ ]:
fig,ax=plt.subplots(figsize=(8,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

for i in range(5):
    ax.plot(ratings_learn_pred[:,i],ratings_learn[:,i],'.',color=learn_colors[i])
ax.plot([-400,500],[-400,500],color='black',linestyle='dashed')
ax.set_xlabel('Predicted Elo')
ax.set_ylabel('Observed Elo')
ax.set_xlim([-400,500])
ax.set_ylim([-400,500])
fig.subplots_adjust(bottom=0.15,right=0.95,left=0.2,top=0.95)
fig.savefig(fig_direc + 'supp_figure_14A.pdf')
plt.show()

In [ ]:
def test_significance(A,x):
    df = pd.DataFrame({'y' : A.flatten(),'subj': np.repeat(range(A.shape[0]),A.shape[1]),'cond' : np.tile(x,A.shape[0])})
    res = sm.ols(formula="y ~ cond + C(subj)",data=df).fit()
    return res.params['cond'],np.mean((res.conf_int(alpha=0.05).loc['cond'].values-res.params['cond'])*[-1,1]),res.pvalues['cond']


In [ ]:
for name,x in zip(['depth','feature drop rate','heuristic quality'],
                  [depth_learn,feature_drop_rate_learn,heuristic_quality_learn]):
    print(name,test_significance(np.mean(x,axis=2),range(5)))


In [ ]:
for name,x in zip(['depth','feature drop rate','heuristic quality'],
                  [depth_time,feature_drop_rate_time,heuristic_quality_time]):
    print(name,test_significance(np.mean(x,axis=2),[5,10,20]))


In [ ]:
for name,x in zip(['depth','feature drop rate','heuristic quality'],
                  [depth_peak,feature_drop_rate_peak,heuristic_quality_peak]):
    print(name,test_significance(np.mean(x,axis=2),range(5)))


In [ ]:
test_significance(ratings_peak,range(5))

In [ ]:
test_significance(ratings_learn,range(5))

In [ ]:
models=['final','final_nonoise','final_nodelta','final_noprune','final_notree','final_noact','final_no3',
        'final_no2conn','final_nocenter','final_no4','final_no2unc',
        'final_optweights','final_mcts_myopic','final_drop_tile','final_fixed_iters',
        'final_fixed_depth','final_fixed_branch','final_weight_hvd','final_drop_hvd',
        'final_triangle','final_drop_type','final_opp']
models=['final','final_nonoise','final_nodelta','final_noprune','final_notree','final_noact','final_no3',
        'final_no2conn','final_nocenter','final_no4','final_no2unc',
        'final_optweights','final_weight_hvd','final_drop_hvd',
        'final_triangle','final_drop_type','final_opp','final_drop_tile',
        'final_fixed_branch','final_fixed_iters','final_fixed_depth']

model_names = ['Main','No value noise','No feature drop','No pruning','No tree','No active scaling',
               'No 3-in-a-row','No connected 2-in-a-row','No center','No 4-in-a-row',
               'No unconnected 2-in-a-row','Optimal weights',
               'Tile dropping','Fixed iterations','Fixed depth','Fixed branching',
               'Orientation-dependent weights','Orientation-dependent dropping','Triangle',
               'Type-dependent dropping','Opponent scaling']


In [ ]:
with open(direc + 'Planning depth/depth/depth_learn_final_fixed_depth_pickle.txt','wb') as f:
    pickle.dump(np.reshape(np.loadtxt(direc + 'Params/params_learn_final_fixed_depth.txt'),[30,5,5,58])[:,:,:,1],file=f)



In [ ]:
def get_depth(m):
    print(m)
    filename = direc + 'Planning depth/depth/depth_learn_' + m + '_pickle.txt'
    if os.path.exists(filename):
        with open(filename,'rb') as f:
            return pickle.load(f)
    return np.reshape([[np.mean(np.loadtxt(direc + 'Planning depth/depth/depth_learn_' + m + '_' + str(i) + '_' + str(j) + '.txt'))
                           for j in range(1,6)] for i in range(0,150)],[30,5,5])
    
depth_learn_models =  [get_depth(m) for m in models]

In [ ]:
import pickle
for d,m in zip(depth_learn_models,models):
    with open(direc + 'Planning depth/depth/depth_learn_' + m + '_pickle.txt','wb') as f:
        pickle.dump(d,file=f)

In [ ]:
heuristic_quality_learn_models = [get_heuristic_quality(np.reshape(np.loadtxt(direc + 'Params/params_learn_' + m + '.txt'),
                                                                   [30,5,5,58])) for m in models]

In [ ]:
feature_drop_learn_models = [np.reshape(np.loadtxt(direc + 'Params/params_learn_' + m + '.txt'),[30,5,5,58])[:,:,:,-15]
                       for m in models]

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

for d in depth_learn_models:
    ax.plot(range(1,6),np.mean(d,axis=(0,2)),'.-',color='black')
ax.set_xlabel('Session')
ax.set_ylabel('Planning depth')
ax.set_xticks(range(1,6))
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
#fig.savefig(fig_direc + 'figure_3b.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

for q in heuristic_quality_learn_models:
    ax.plot(range(1,6),np.mean(q,axis=(0,2)),'.-',color='black')
ax.set_xlabel('Session')
ax.set_ylabel('Heuristic quality')
ax.set_xticks(range(1,6))
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
#fig.savefig(fig_direc + 'figure_3b.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(6,6))
ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))

for f in feature_drop_learn_models:
    ax.plot(range(1,6),np.mean(f,axis=(0,2)),'.-',color='black')
ax.set_xlabel('Session')
ax.set_ylabel('Feature drop rate')
ax.set_xticks(range(1,6))
fig.subplots_adjust(bottom=0.2,right=0.95,left=0.25)
#plt.text(0,0.9,'A',ha='right',va='top',transform=fig.transFigure)
#fig.savefig(fig_direc + 'figure_3b.pdf')
plt.show()

In [ ]:
for d,q,f,m in zip(depth_learn_models,heuristic_quality_learn_models,feature_drop_learn_models,models):
    print(m)
    slopes = [get_slopes(ratings_learn[:,i],np.mean(d,axis=2)[:,i],np.mean(f,axis=2)[:,i],np.mean(q,axis=2)[:,i])
                     for i in range(5)]
    ratings_learn_pred = np.mean(np.sum([s*A for s,A in zip(np.mean(slopes,axis=0),[d,f,q])],axis=0),axis=2)
    ratings_learn_pred += np.mean(slopes,axis=0)[-1]
    for s,x in zip(np.mean(slopes,axis=0),[d,f,q]):
        print(s*np.mean(x[:,-1]-x[:,0]),np.abs(s)*np.std(np.mean(x[:,-1,:]-x[:,0,:],axis=1))/np.sqrt(x.shape[0]))

In [ ]:
def report_stats(s):
    rho,p = s
    if np.isnan(p):
        return '\\multicolumn{2}{c |}{N/A}'
    elif p>0.01:
        return '$' + '{0:.2f}'.format(rho) + '$ & $' + '{:.2f}'.format(p) + '$'
    else:
        return '$' + '{0:.2f}'.format(rho) + '$ & $' + '{:.1e}'.format(p).split('e')[0] + '\cdot 10^{' + str(int('{0:.1e}'.format(p).split('e')[1])) + '}$'

for d,q,f,m in zip(depth_learn_models,heuristic_quality_learn_models,feature_drop_learn_models,model_names):
    print(m,'&',report_stats(sts.pearsonr(np.mean(d,axis=2).flatten(),ratings_learn.flatten())),'&',
          report_stats(sts.pearsonr(np.mean(f,axis=2).flatten(),ratings_learn.flatten())),'&',
          report_stats(sts.pearsonr(np.mean(q,axis=2).flatten(),ratings_learn.flatten())),'\\\\\hline')


In [ ]:
depth_hvh_models = [np.array([[np.mean(np.loadtxt(direc + 'Planning depth/depth/depth_learn_' + m + '_' + str(i) + '_' + str(j) + '.txt')) 
                                for j in range(1,6)] for i in range(40)]) for m in ['final','final_notree','final_nodelta','final_optweights']]

In [ ]:
feature_drop_hvh_models = [np.reshape(np.loadtxt(direc + 'Params/params_hvh_' + m + '.txt'),[40,5,58])[:,:,-15]
                       for m in ['final','final_notree','final_nodelta','final_optweights']]


In [ ]:
heuristic_quality_hvh_models = [get_heuristic_quality(np.reshape(np.loadtxt(direc + 'Params/params_hvh_' + m + '.txt'),
                                                                   [40,1,5,58]))[:,0,:] for m in ['final','final_notree','final_nodelta','final_optweights']]

In [ ]:
Mfull = np.array([np.mean(depth_hvh_models[0],axis=1),
         np.mean(feature_drop_hvh_models[0],axis=1),
         np.mean(heuristic_quality_hvh_models[0],axis=1)])

Mlesions = np.array([[np.mean(depth_hvh_models[i],axis=1),
         np.mean(feature_drop_hvh_models[i],axis=1),
         np.mean(heuristic_quality_hvh_models[i],axis=1)] for i in range(1,4)])


In [ ]:
def get_alpha_fdr(alpha,allps):
    return allps[np.nonzero(allps > np.arange(1,len(allps)+1)/len(allps)*alpha)[0][0]]

def plot_with_stars(r,p,colormap,limits,filename):
    fig,ax = plt.subplots(figsize=(12,12))
    ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
    ax.yaxis.set_major_formatter(FormatStrFormatter('%g'))
    
    nstars = (p < get_alpha_fdr(0.05,np.sort(p.flatten()))).astype(int) + \
             (p < get_alpha_fdr(0.01,np.sort(p.flatten()))).astype(int) + \
             (p < get_alpha_fdr(0.001,np.sort(p.flatten()))).astype(int)
        
    for i in range(r.shape[0]):
        for k in range(r.shape[1]):
            if k!=i and nstars[i][k]>0:
                ax.text(k,i,'*'*nstars[i][k],ha='center',va='center',fontsize=24)
    
    ax.set_xticks(range(3))
    ax.set_yticks(range(3))
    ax.set_xticklabels(['Planning depth','Feature drop rate','Heuristic quality'],usetex=False,rotation=60,ha='right',
                       fontsize = 32)
        
    ax.set_yticklabels(['Planning depth','Feature drop rate','Heuristic quality'],usetex=False,fontsize = 32)    
    cax = ax.imshow(r,interpolation='nearest',cmap=colormap,vmin=limits[0],vmax=limits[1])
    cbar = fig.colorbar(cax, orientation='vertical',ticks=[-1, 0, 1])
    cbar.ax.set_yticklabels([-1,0,1],usetex=False)
    fig.subplots_adjust(bottom=0.3,right=0.95,left=0.35,top=0.9)    
    fig.savefig(fig_direc + filename)
    plt.show()

In [ ]:
rho1 = np.array([[sts.pearsonr(Mfull[i,:]-Mlesions[j,i,:],Mfull[j,:])[0] for i in range(3)] for j in range(3)])
P1 = np.array([[sts.pearsonr(Mfull[i,:]-Mlesions[j,i,:],Mfull[j,:])[1] for i in range(3)] for j in range(3)])

In [ ]:
rho1

In [ ]:
plot_with_stars(rho1,P1,'bwr',[-1,1],'supp_figure_7B.pdf')

In [ ]:
D2 = np.array([[sts.ks_2samp(Mfull[i,:],Mlesions[j,i,:])[0] for i in range(3)] for j in range(3)])
P2 = np.array([[sts.ks_2samp(Mfull[i,:],Mlesions[j,i,:])[1] for i in range(3)] for j in range(3)])

In [ ]:
plot_with_stars(D2,P2,'Greens',[0,1],'supp_figure_7A.pdf')